In [1]:
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

df_raw = pd.read_csv("BASE_1_Manga_Produtos.csv")
df = df_raw.copy()

missing_mask = df.isna()

# Indicadores de missing (boa prática p/ MAR/MNAR)

for c in df.columns:
    if df[c].isna().any():
        df[f"miss__{c}"] = df[c].isna().astype(int)

# 3) Tratamento por regra (empresa -> faturamento) + FALLBACKS

# 3.1) Primeiro: por empresa (mais consistente)
df["faturamento_empresa_brl"] = df["faturamento_empresa_brl"].fillna(
    df.groupby("empresa")["faturamento_empresa_brl"].transform("median")
)

# 3.2) Fallback: por (mercado_principal, tipo_cliente)
df["faturamento_empresa_brl"] = df["faturamento_empresa_brl"].fillna(
    df.groupby(["mercado_principal", "tipo_cliente"])["faturamento_empresa_brl"].transform("median")
)

# 3.3) Fallback: por mercado_principal
df["faturamento_empresa_brl"] = df["faturamento_empresa_brl"].fillna(
    df.groupby("mercado_principal")["faturamento_empresa_brl"].transform("median")
)

# 3.4) Fallback final: mediana global
df["faturamento_empresa_brl"] = df["faturamento_empresa_brl"].fillna(
    df["faturamento_empresa_brl"].median()
)

# 4) Categóricas: moda por grupo -> fallback "Nao informado"
cat_cols = [
    "empresa", "produto", "tipo_cliente",
    "mercado_principal", "certificacoes_principais"
]

def fill_mode_by_group(data: pd.DataFrame, target: str, group_cols: list[str]) -> pd.DataFrame:
    """Preenche NaN em target com a moda dentro de cada grupo definido por group_cols."""
    mode_map = (
        data.dropna(subset=[target])
            .groupby(group_cols)[target]
            .agg(lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan)
    )

    idx = data[target].isna()
    if idx.any():
        keys = list(zip(*[data.loc[idx, g] for g in group_cols]))
        data.loc[idx, target] = [mode_map.get(k, np.nan) for k in keys]
    return data

for c in cat_cols:
    df = fill_mode_by_group(df, c, ["empresa", "produto"])
    df = fill_mode_by_group(df, c, ["empresa"])
    df = fill_mode_by_group(df, c, ["produto"])
    df[c] = df[c].fillna("Nao informado")

# 5) Numéricas: imputação multivariada (MAR, cobre MCAR)
num_cols = [
    "indice_concorrencia_1a5",
    "indice_valor_agregado_1a5",
    "margem_media_percentual",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5",
]

imp = IterativeImputer(
    estimator=BayesianRidge(),
    random_state=42,
    max_iter=25,
    sample_posterior=True,
    skip_complete=True
)

df[num_cols] = imp.fit_transform(df[num_cols])

# 6) Corrigir domínio dos índices (1..5 e 0.5 no valor agregado)

int_indices = [
    "indice_concorrencia_1a5",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5",
]
df[int_indices] = df[int_indices].round().clip(1, 5)

df["indice_valor_agregado_1a5"] = (df["indice_valor_agregado_1a5"] * 2).round() / 2
df["indice_valor_agregado_1a5"] = df["indice_valor_agregado_1a5"].clip(1, 5)

df["margem_media_percentual"] = df["margem_media_percentual"].clip(lower=0)



total_missing_final = int(df.isna().sum().sum())
print("Total de NaNs após tratamento:", total_missing_final)

imputed_report = pd.DataFrame({
    "missing_original": missing_mask.sum(),
    "missing_final": df.isna().sum()
}).sort_values("missing_original", ascending=False)

print(imputed_report.head(20))

# =========================
# 8) Salvar base tratada
# =========================
df.to_csv("BASE_1_Manga_Produtos_TRATADA.csv", index=False)
print("Arquivo salvo: BASE_1_Manga_Produtos_TRATADA.csv")


Total de NaNs após tratamento: 0
                                                   missing_original  \
indice_valor_agregado_1a5                                      98.0   
produto                                                        94.0   
empresa                                                        91.0   
indice_capex_1a5                                               91.0   
tipo_cliente                                                   90.0   
indice_escalabilidade_1a5                                      90.0   
indice_compatibilidade_refugo_1a5                              87.0   
indice_concorrencia_1a5                                        84.0   
mercado_principal                                              80.0   
margem_media_percentual                                        79.0   
certificacoes_principais                                       79.0   
indice_complexidade_produtiva_1a5                              78.0   
faturamento_empresa_brl                     

Tratamento de dados faltantes: Inicialmente, foram criados indicadores binários de ausência (miss__coluna) para registrar onde existiam valores faltantes. Em seguida, o faturamento_empresa_brl foi imputado de forma condicional, priorizando a mediana por empresa e, quando necessário, aplicando uma cascata de fallback por grupos de negócio (mercado_principal e tipo_cliente) e por mercado_principal, com mediana global como último recurso. Para variáveis categóricas, os valores faltantes foram preenchidos pela moda por grupo (empresa+produto → empresa → produto), usando “Não informado” como fallback quando não havia informação suficiente. Já as variáveis numéricas (índices 1–5 e margem) foram imputadas via imputação multivariada (IterativeImputer/MICE-like), preservando as relações entre variáveis; ao final, os índices foram ajustados para respeitar seu domínio (1–5, com passos de 0,5 para valor agregado). Após o processo, a base ficou sem valores ausentes.

In [2]:
import numpy as np

#1..5
int_cols = [
    "indice_concorrencia_1a5",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5",
]

print("Algum índice fora de 1..5?", ((df[int_cols] < 1) | (df[int_cols] > 5)).any().any())

# valor agregado em 0.5
va = df["indice_valor_agregado_1a5"]
print("Valor agregado fora do domínio?", ((va < 1) | (va > 5)).any())
print("Valor agregado não múltiplo de 0.5?", ((va * 2) % 1 != 0).any())

# margem
print("Margem negativa?", (df["margem_media_percentual"] < 0).any())

# faturamento
print("Faturamento <= 0?", (df["faturamento_empresa_brl"] <= 0).any())


Algum índice fora de 1..5? False
Valor agregado fora do domínio? False
Valor agregado não múltiplo de 0.5? False
Margem negativa? False
Faturamento <= 0? False


In [3]:
import pandas as pd

df_before = df_raw.copy() # base original
df_after = df.copy() # base tratada
cols_check = [
    "margem_media_percentual",
    "indice_valor_agregado_1a5",
    *int_cols
]

for c in cols_check:
    m = df_before[c].isna()
    if m.sum() == 0:
        continue
    print("\n===", c, "===")
    print("Qtd imputada:", int(m.sum()))
    print("Antes (observado) - média/mediana:",
          df_before.loc[~m, c].mean(), df_before.loc[~m, c].median())
    print("Depois (imputado) - média/mediana:",
          df_after.loc[m, c].mean(), df_after.loc[m, c].median())



=== margem_media_percentual ===
Qtd imputada: 79
Antes (observado) - média/mediana: 28.353275755706356 27.93
Depois (imputado) - média/mediana: 28.766551160331158 29.73074189643633

=== indice_valor_agregado_1a5 ===
Qtd imputada: 98
Antes (observado) - média/mediana: 2.9060549313358304 3.0
Depois (imputado) - média/mediana: 2.8418367346938775 2.5

=== indice_concorrencia_1a5 ===
Qtd imputada: 84
Antes (observado) - média/mediana: 2.9832920792079207 3.0
Depois (imputado) - média/mediana: 3.0476190476190474 3.0

=== indice_complexidade_produtiva_1a5 ===
Qtd imputada: 78
Antes (observado) - média/mediana: 3.0437731196054254 3.0
Depois (imputado) - média/mediana: 2.9358974358974357 3.0

=== indice_capex_1a5 ===
Qtd imputada: 91
Antes (observado) - média/mediana: 2.8228713486637664 3.0
Depois (imputado) - média/mediana: 2.8351648351648353 3.0

=== indice_escalabilidade_1a5 ===
Qtd imputada: 90
Antes (observado) - média/mediana: 3.0372670807453415 3.0
Depois (imputado) - média/mediana: 3.12

In [4]:
import numpy as np

num_cols = [
    "faturamento_empresa_brl",
    "margem_media_percentual",
    "indice_valor_agregado_1a5",
    *int_cols
]

corr_before = df_raw[num_cols].corr(numeric_only=True)
corr_after  = df_after[num_cols].corr(numeric_only=True)

diff = (corr_after - corr_before).abs()

print("Maiores mudanças de correlação (top 10):")
pairs = diff.where(np.triu(np.ones(diff.shape), k=1).astype(bool)).stack().sort_values(ascending=False)
print(pairs.head(10))


Maiores mudanças de correlação (top 10):
indice_complexidade_produtiva_1a5  indice_exigencia_regulatoria_1a5               0.015386
                                   indice_compatibilidade_operacao_fazenda_1a5    0.013275
margem_media_percentual            indice_capex_1a5                               0.011800
indice_capex_1a5                   indice_compatibilidade_refugo_1a5              0.011304
margem_media_percentual            indice_complexidade_produtiva_1a5              0.011175
indice_escalabilidade_1a5          indice_compatibilidade_refugo_1a5              0.010636
faturamento_empresa_brl            margem_media_percentual                        0.010543
indice_complexidade_produtiva_1a5  indice_capex_1a5                               0.010107
margem_media_percentual            indice_exigencia_regulatoria_1a5               0.009816
indice_complexidade_produtiva_1a5  indice_escalabilidade_1a5                      0.008906
dtype: float64


In [5]:
cols = [
    "margem_media_percentual",
    "indice_valor_agregado_1a5",
    "indice_concorrencia_1a5",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5",
]

for c in cols:
    m = df_raw[c].isna()
    if m.sum() == 0:
        continue
    print("\n", c)
    print("STD observado:", df_raw.loc[~m, c].std())
    print("STD imputado :", df.loc[m, c].std())



 margem_media_percentual
STD observado: 10.709330860174099
STD imputado : 11.378865202474529

 indice_valor_agregado_1a5
STD observado: 0.8048538211581536
STD imputado : 0.9041625925065672

 indice_concorrencia_1a5
STD observado: 1.1437131658150064
STD imputado : 1.063125216811627

 indice_complexidade_produtiva_1a5
STD observado: 1.1206909685879753
STD imputado : 1.0486818391359698

 indice_capex_1a5
STD observado: 1.2279189850359327
STD imputado : 1.195024279286002

 indice_escalabilidade_1a5
STD observado: 1.1391021660318037
STD imputado : 1.0791983893360702

 indice_exigencia_regulatoria_1a5
STD observado: 1.1863225576944674
STD imputado : 1.1167942185170971

 indice_compatibilidade_refugo_1a5
STD observado: 1.2564399847824137
STD imputado : 1.0245581127409613

 indice_compatibilidade_operacao_fazenda_1a5
STD observado: 1.2631995236359355
STD imputado : 1.245707237650876
